In [ ]:
import torch
import pandas as pd
from sklearn.metrics import precision_score, recall_score, f1_score
from scipy.stats import mode
from transformers import BertTokenizer
from transformers import BertForSequenceClassification, Trainer, TrainingArguments
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from datasets import Dataset
from sklearn.metrics import classification_report, accuracy_score, precision_recall_fscore_support

In [ ]:
comments

,index,weibo_id,comment_text,gender,location,like,date,label,category,target
0,0,J1zF7smOR,她们不结婚不要孩子，却觉得我们这种结婚有孩子的是婚驴,女,广东,14,12/05/2020 15:04,1,SCB,G
1,1,GvXf0ymdJ,妈的什么时候有人来性侵我一下啊 不要封口费的那种,男,其他,36,23/08/2018 13:10,1,SO,I
2,2,J2XeR8rPn,我觉得这就是对一类人所有的特点进行形容吧，“爹味”也是贬义词，为什么不觉得有性别歧视意味呢？,女,浙江,0,21/05/2020 16:12,1,MA,G
3,3,IxpfTCMAo,姐姐你不要急，关注久的都知道她骂的一切归根结底都是渴婚的，她并不骂学艺术的，她骂的是不肯像培...,女,江苏,1,31/03/2020 13:57,1,MA,I
4,4,J4B0Mps5Q,"给他们争取权利的不是女权是腐女，你该不会说腐女都女权吧？连女同性恋都不是全部女权，腐女？,男...",女,其他,0,##########,1,MA,G
...,...,...,...,...,...,...,...,...,...,...
8964,8964,GxUGteFvS,之前一直是五个男生那种，你们批评我吧,男,广东,0,05/09/2018 12:04,0,NaN,NaN
8965,8965,Fq0uZhLsx,中国的小孩也可以随母亲姓啊 我一个朋友就是随母亲姓的,男,其他,0,12/10/2017 11:50,0,NaN,NaN
8966,8966,H5OE5y1ij,所有为韦涛卖命而去攻击别人的炮灰都快点死.......咒语灵灵灵。。。。。。,男,广东,2,05/12/2018 17:50,0,NaN,NaN
8967,8967,J2cWRaHx1,本质是阶级矛盾……比如你是个资本家，别人抱你大腿的时候就不会管你是男还是女,男,上海,1,16/05/2020 17:56,0,NaN,NaN


In [ ]:
train_texts, val_texts, train_labels, val_labels = train_test_split(
    comments["comment_text"].tolist(), comments["label"].tolist(), test_size=0.2, random_state=42
)

tokenizer = BertTokenizer.from_pretrained("bert-base-chinese")

def tokenize_function(texts):
    return tokenizer(texts, padding="max_length", truncation=True, max_length=128)

train_encodings = tokenize_function(train_texts)
val_encodings = tokenize_function(val_texts)

train_dataset = Dataset.from_dict({
    "input_ids": train_encodings["input_ids"],
    "attention_mask": train_encodings["attention_mask"],
    "labels": train_labels
})

val_dataset = Dataset.from_dict({
    "input_ids": val_encodings["input_ids"],
    "attention_mask": val_encodings["attention_mask"],
    "labels": val_labels
})

model = BertForSequenceClassification.from_pretrained("bert-base-chinese", num_labels=2)

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="binary")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=2,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="f1"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()
eval_results = trainer.evaluate()
eval_results

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-chinese and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: snw2136 (snw2136-columbia-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.481100,0.437030,0.770903,0.617044,0.910828,0.735691
2,0.401500,0.436597,0.797659,0.719735,0.691083,0.705118


{'eval_loss': 0.4370296001434326,
 'eval_accuracy': 0.7709030100334449,
 'eval_precision': 0.6170442286947141,
 'eval_recall': 0.910828025477707,
 'eval_f1': 0.7356913183279743,
 'eval_runtime': 12.229,
 'eval_samples_per_second': 146.701,
 'eval_steps_per_second': 4.661,
 'epoch': 2.0}

In [ ]:
eval_results

{'eval_loss': 0.4370296001434326,
 'eval_accuracy': 0.7709030100334449,
 'eval_precision': 0.6170442286947141,
 'eval_recall': 0.910828025477707,
 'eval_f1': 0.7356913183279743,
 'eval_runtime': 12.229,
 'eval_samples_per_second': 146.701,
 'eval_steps_per_second': 4.661,
 'epoch': 2.0}

In [ ]:
preds_output = trainer.predict(val_dataset)
preds = preds_output.predictions.argmax(-1)
true = preds_output.label_ids

val_df = pd.DataFrame({
    "text": val_texts,
    "true_label": true,
    "pred_label": preds
})

fp_df = val_df[(val_df["true_label"] == 0) & (val_df["pred_label"] == 1)]
fn_df = val_df[(val_df["true_label"] == 1) & (val_df["pred_label"] == 0)]

pd.set_option('display.max_colwidth', None)
print("False Positives (predicted misogyny, actually neutral):")
display(fp_df.sample(5, random_state=1))

print("False Negatives (predicted neutral, actually misogyny):")
display(fn_df.sample(5, random_state=1))

False Positives (predicted misogyny, actually neutral):


,text,true_label,pred_label
1738,私信2就算你去当失足妇女也还不完你爸欠的赌债，年纪越大皮肉越不值钱，相反你爸反而可以年纪越大赌钱输得越多,0,1
757,我记得这是违法的，怎么就没人管了？当年我们的政治课上有一个案例就是父母逼婚，被判三年的，女权关注和转发这类的事情非常多，不能您看不见就cui女权啊,0,1
435,一天跟跳梁小丑似的，她们爱女人，我可没那爱好，我这人很挑剔的，我只爱大帅哥,0,1
945,一转眼，MeToo运动一周年了。我们跟昨天节目里的的上天入地女导演申迪老师聊了聊一个女性眼中的me too，以及她亲身经历的2段故事和不同的处理方式。哈哈哈，终于不是俩直男聊这个了～,0,1
485,人家霸道男主是喜欢女主才那样的啊 算不上性犯罪吧,0,1


False Negatives (predicted neutral, actually misogyny):


,text,true_label,pred_label
1415,她们焦虑自己，同时也将焦虑施压于其他人，对颜值身材高度量化细化。不过如果我现在是20岁的话很可能也会受影响。,1,0
149,哈哈，别生气，她就是个哗众取宠的小丑罢了，古人诚不欺我，这个成语用在这里太合适不过了,1,0
1464,"我觉得你说的都有道理，但我之前看有博主反思那些为柯洁辩解的人的话语－－就是所谓“只是个孩子”云云，她说这是很强烈的“妈感”，就像妈妈看犯错的孩子一样去宽容、去理解孩子的错误。这种想法在争论中并没有任何说服力，因为这类话语只代表说话人对柯洁本人的宽容态度而已，没有任何逻辑上的力量。,就比如他攻击那些所谓田园女权的女生，他完全可以以理服人，用她们的逻辑漏洞来反驳，这样完全会赢得大方得体。但他没有，除了一味写一些乱七八糟的emoji、跟着网友一起骂140斤的女生是“坦克”以外就没有了。我不认为真女权能够对任何一位意见与其相左的人发起人身攻击。,我以前也觉得是田园女权要搞他，但看了他恶臭言论合集以后我真的不敢相信有任何一位真正的女权主义者能够在任何情况下说出那种话",1,0
888,对啊一个巴掌拍不响这种早已经过时了好吗兄弟 不自重和性侵是一码事吗？,1,0
1209,"那现在网游做爆乳妹子的也很多啊,视野开阔点嘛",1,0


In [ ]:
#similar with the conclusion with clustering, there seems to be some mislabelled texts to begin with, where the SWSR dataset actually does not provide us with an accurate labelling

In [ ]:
tp_df = val_df[(val_df["true_label"] == 1) & (val_df["pred_label"] == 1)]
tn_df = val_df[(val_df["true_label"] == 0) & (val_df["pred_label"] == 0)]

pd.set_option('display.max_colwidth', None)
print("True Positives (predicted misogyny, actually misogyny):")
display(tp_df.sample(5, random_state=1))

print("True Negatives (predicted neutral, actually neutral):")
display(tn_df.sample(5, random_state=1))

True Positives (predicted misogyny, actually misogyny):


,text,true_label,pred_label
1395,她说的太娘了是指她自己太弱了吧，枪的后座力度大吓到她自己了，她觉得丢脸啦吧,1,1
1509,其实我理解是极端派只是想改良法律，在婚姻制度里面偏斜女性多点就行，但要废除婚姻制度基本是妄想所以只能自身做起不结婚,1,1
1711,哇？？？？？？？吐槽鬼居然是女性我真的觉得她可爱了哈哈哈哈哈,1,1
1604,没有女性车厢，它们咬人；有女性车厢，它们还咬人；而且咬人的还是愿班人马。以后别伺候这班大爷，社会关爱女性是必须的，但别取悦女权，更别配合它们打着扛着女权大旗迫害女性。,1,1
515,现在即使是博主所说的女权也会被骂女👊,1,1


True Negatives (predicted neutral, actually neutral):


,text,true_label,pred_label
831,由于爱惜羽毛的缘故 从不在网上和别人争论 不企图改变一个陌生人的想法 我的话只说给愿意听的人听,0,0
685,"那就吃瓜看戏啊。被抓进去两个月了，如果没证据，早该出来了。吴秀波也没什么好结果，节目下架了。,她敲诈如果是事实的话，逼急了，狗都跳墙。",0,0
733,和搞文字狱一样，抠字眼找问题，自以为找到了就要把人拖出来枪决,0,0
1296,好看，没有撕心裂肺的言明爱情，但每一个人都用心的去相爱，尽管生活有太多的无奈。值得去看。,0,0
18,"果然有一条被pb…我也觉得他现在这样就这样了，以后可能还会被触动被温暖但是没有办法让铁铁成为普世眼中的健康的人。他小时候的事情不说，作为铁铁他就生活在一个普世无法理解的世界里，他所看到的所经历的事情对他造成的影响是不可逆也无法被治愈的。但我还是想要看到有人愿意爱他，不求回报地爱他 评论配图,jj和铁的背景差别太大了，jj有时候给我感觉就是已经懒得去put up a fight这样，当然这也有故事格局的因素，但是铁铁的upbringing就决定了，有时候我感觉他其实就是追求利益最大化，包括一些自我牺牲的行为，他是将一切条件都资本化的感觉，materially burdened it is，做铁真的好累了",0,0


In [ ]:
#from what can be seen, there are some texts that the model identifies better than the original label. So I will take a better look at the training dataset and see if there are major mistakes
train = pd.DataFrame(train_texts, train_labels).reset_index()
train.columns = ['label', 'text']
train.head(50)

,label,text
0,0,其实我觉得也不能太偏激了吧。我们男性不说不代表我们不知道对错，只是不喜欢去评论这些事情。
1,0,不完全统计，十三个伏地魔相关博主被炸号，其中包括一位维权素人
2,0,只是从图二里表达出来的是那些发达国家，我也没有不尊重其他国家，只是觉得一味地崇洋媚外，甚至说“中国的基因丑”，真的挺过分的
3,0,其他的不说 对待舆论的态度真的圈粉 不卑不亢 掷地有声💚
4,1,男人也吃男人，也有男吃女女吃男，怎么就毫无存在感了？单独拿出来说女吃女，仿佛是为了证明女性的劣根性，有没有欺压都他妈只会窝里斗？欺压还分出个三六九等来了？
5,0,一场运动已经变质了把人整死了，大规模动作翻老底?整个韩娱蔓延到爱豆圈搞得人心惶惶不少韩国男性出现“恐女症”，
6,0,他太玩不起了，只敢说脏话不敢对线，还说自己射爹呢，防御塔都比他刚
7,0,对的，特别是我和母亲关系非常好，我母亲是个特别好的人。再加上我身边的女孩都特别善良
8,0,"嗯，不同理解角度，大家多听多看多思考吧，也不能都让情绪做主导,后面的其实不是在问，是在倾诉，很多一看就是发问者的亲身经历"
9,0,没办法，既然非要搞人吃人的丛林法则逻辑，那他们被淘汰也是活该


In [ ]:
#there are some false positives here, which indicates that the original labelling might not have considered nuances/connotations/context when labelling. There are some keywords that may have indicated that it is misogynistic. Which leads to me to think if it would be more reasonable to find terms as units of misogyny and track it that way? (doing it the lexicon way which the method also proposes)